<a href="https://colab.research.google.com/github/irina1nik/ai_notebooks/blob/main/index_from_website_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook makes a question answering chain with a specified website as a context data.

# Setting up

Install dependencies

In [ ]:
!pip install langchain==0.0.189
!pip install pinecone-client
!pip install openai
!pip install tiktoken
!pip install nest_asyncio

Set up OpenAI API key

In [ ]:
import os 
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

Set up Pinecone API keys

In [ ]:
import pinecone 

# initialize pinecone
pinecone.init(
    api_key="PINECONE_API_KEY",  # find at app.pinecone.io
    environment="ENVIRONMENT_NAME"  # next to api key in console
)

# Index 

**Load data from Web**

Extends from the WebBaseLoader, this will load a sitemap from a given URL, and then scrape and load all the pages in the sitemap, returning each page as a document.

The scraping is done concurrently, using WebBaseLoader. There are reasonable limits to concurrent requests, defaulting to 2 per second.

Link to the [documentation](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/sitemap.html)

In [ ]:
# fixes a bug with asyncio and jupyter
import nest_asyncio
nest_asyncio.apply()

from langchain.document_loaders.sitemap import SitemapLoader

loader = SitemapLoader(
    "https://ind.nl/sitemap.xml",
    filter_urls=["https://ind.nl/en"]
)
docs = loader.load()

Fetching pages: 100%|##########| 505/505 [03:19<00:00,  2.53it/s]


**Split the text from docs into smaller chunks**

There are many ways to split the text. We are using the text splitter that is recommended for generic texts. For more ways to slit the text check the [documentation](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1200,
    chunk_overlap  = 200,
    length_function = len,
)

docs_chunks = text_splitter.split_documents(docs)

Create embeddings

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

**Creating a vectorstore**

A vectorstore stores Documents and associated embeddings, and provides fast ways to look up relevant Documents by embeddings. 

There are many ways to create a vectorstore. We are going to use Pinecone. For other types of vectorstores visit the [documentation](https://python.langchain.com/en/latest/modules/indexes/vectorstores.html)

First you need to go to [Pinecone](https://www.pinecone.io/) and create an index there. Then type the index name in "index_name"

In [ ]:
from langchain.vectorstores import Pinecone

index_name = "ind"

# #create a new index
docsearch = Pinecone.from_documents(docs_chunks, embeddings, index_name=index_name)

# if you already have an index, you can load it like this
#docsearch = Pinecone.from_existing_index(index_name, embeddings)


If you cannot create a Pinecone account, try to use CromaDB. The following code creates a transient in-memory vectorstore. For further information check the [documentation](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/chroma.html). 

The following code block uses Croma for creating a vectorstore. Uncomment it if you don't have access to pinecone and use it instead.

In [ ]:
# from langchain.vectorstores import Chroma
# docsearch = Chroma.from_documents(docs, embeddings)

Vectorstore is ready. Let's try to query our docsearch with similarity search

In [ ]:
query = "How to get a visa for my partner if I have a highly skilled visa."
docs = docsearch.similarity_search(query)
print(docs[0])

# Making a question answering chain
The question answering chain will enable us to generate the answer based on the relevant context chunks. See the [documentation](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html) for more explanation.

Additionally, we can return the source documents used to answer the question by specifying an optional parameter when constructing the chain. For more information visit the [documentation](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html#return-source-documents)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
llm=OpenAI()

qa_with_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)

query = "How to get a visa for my partner if I have a highly skilled visa."
result = qa_with_sources({"query": query})
result["result"]

Output source documents that were found for the query

In [ ]:
result["source_documents"]